Суть задания в том, чтобы выкинуть ненужные картинки и заменить описание у нужных картинок, при его наличии.

Изначально будет несколько архивов. В каждом архиве есть папка с фото, называется 0.tar, 1.tar и тд, к каждому такому тарнику будет идти цсвшка с описанием фотографий, называются аналогично 0.csv, 1.csv и так далее. 0й тарник соответствует нулевой цсвшке и далее по аналогии. Такие цсвшки назову основными.

Также в архиве будут цсвшки с названием 0_new.csv, 1_new.csv и так далее. Это те же самые цсвшки, но в них подправлены описания и есть доп колонки. Такие цсвшки назову дополнительными

Алгоритм:
1. сначала нужно выкинуть из основных цсвшек те фотки, у которых в дополнительных в колонке query_flags стоит 1. в качестве ключа берите первую колонку image_name, это название самой фотки
2. затем нужно заменить описания в основной цсвшке на описания из дополнительной для оставшихся фото. описания к фото записаны в колонке caption
3. после этого на выходе вы получите несколько цсвшек, в которых исправленные описания и часть фото удалены. те фото, которых нет в цсвшках нужно удалить из самих архивов с фото
4. теперь нужно пересобрать цсвшки и тарники так, чтобы там было по 1000 фотографий. только у последней цсвшки в списке и у последнего соответствующего ему тарника может быть меньше 1000 фоток. рекомендую к названию фото просто добавить номер папки, откуда это фото, в стиле 0.png станет 0_0.png, если это фото из нулевого тарника например. название самого фото меняем и название фото в цсвшке, соответственно
5. результат залить на s3 или пинговать меня, дам диск куда лить

Что может пойти не так и на что нужно обратить внимание:
1) ребята-разметчики некоторые изнасиловали цсвшки и где-то в дополнительных цсвшках разделитель запятая, где-то точка с запятой, где-то флаг имеет тип строка, где-то инт, где-то флоат (вообще по умолчанию он был интом). придется немного посидеть над тем, чтобы все это корректно считать. если совсем никак - пингуйте меня
2) перед тем как удалять строки из цсвшек или архивов - сделайте резервную копию или сделайте отдельно тестовый блок себе. мне будет проблематично вам еще раз данные отсылать или вам их снова качать
3) когда вы делаете тарник (его можно сделать хоть консолькой) смотрите, чтобы он не создавал архив в архиве. то есть при разорхивации тарника 0.tar должны появляться сразу фото, а не папка 0, в которой уже лежат фото


In [2]:
import pandas as pd
import os
import cv2
import shutil
import numpy as np
from collections import Counter
import tarfile
import tarfile, glob

## Main part

In [3]:
dir_name = 'C:/Users/annai/Filter_new_csv/data_30_59'
dir_save = 'C:/Users/annai/Filter_new_csv/rename'

shard_idx = 0

file_data = str(shard_idx) + '.csv'
new_data = str(shard_idx) + '_new.csv'

csv_path = os.path.join(dir_name,file_data)
new_data_path = os.path.join(dir_name,new_data)


df = pd.read_csv(csv_path)


In [4]:
df.head()

,image_name,caption,width,height,url,LANGUAGE,similarity,punsafe,pwatermark,hash
0,0.jpg,Дуговой фрезерный нож резак для дерева фрезы б...,800,771,https://ae01.alicdn.com/kf/HTB1p6KqRFXXXXceXVX...,ru,0.264643,4.679442e-05,0.248935,720572426622240968
1,1.jpg,"Баня на реке и на берегу реки: преимущества, н...",1280,853,https://par-torg.com/wp-content/uploads/c/9/e/...,ru,0.266469,1.565069e-03,0.240798,-3561693715683967203
2,2.jpg,"""""""Ульяновская область стала родиной Колобка ...",960,720,https://ds02.infourok.ru/uploads/ex/090e/00000...,ru,0.278951,4.573132e-01,0.276760,-8368125490294308610
3,3.jpg,"замок зажигания уаз буханка: устройство, схема...",1280,720,https://nashikolesa.ru/wp-content/uploads/2020...,ru,0.278530,3.395230e-03,0.217097,-5736097797218125412
4,4.jpg,Открытый Сенсор светодиодный прожекторы IP65 1...,1000,1000,https://ae01.alicdn.com/kf/HTB1J_dEJVXXXXa1XVX...,ru,0.288629,5.561715e-07,0.117737,-3381977850604610995


In [27]:
df_new_data = pd.read_csv(new_data_path, sep=';',encoding='cp1251', on_bad_lines='skip')

In [28]:
df_new_data.head()

,image_name,caption,query_flags
0,0.jpg,Деревообрабатывающий дуговой фрезерный нож,0
1,1.jpg,Баня плывёт по воде,0
2,2.jpg,"""""""Ульяновская область стала родиной Колобка ""...",1
3,3.jpg,Замок зажигания УАЗ,0
4,4.jpg,Чёрный настенный светодиодный прожектор,0


In [273]:
nan_idx_q = np.where(df_new_data['query_flags'].isnull())[0]

In [274]:
nan_idx_q

array([], dtype=int64)

In [275]:
print(len(df_new_data))
print(len(df))

1000
1000


In [276]:
df_new_data.tail()

,image_name,caption,query_flags
995,4995.jpg,Книга театр теней Дюймовочка,0
996,4996.jpg,Прозрачный больше сетка пластик в коробку милы...,0
997,4997.jpg,Djeco Музыкальный инструмент Крокодил06372Музы...,0
998,4998.jpg,Тетрациклиновые зубы – как избавиться от этой ...,0
999,4999.jpg,Красивые шарики на день рождения для девочки 01,0


1. сначала нужно выкинуть из основных цсвшек те фотки, у которых в дополнительных в колонке query_flags стоит 1. в качестве ключа берите первую колонку image_name, это название самой фотки
df_new_datanan_idx = np.where(df['column_name'].isnull())[0]

In [277]:
#получить индекс строки, где хотя бы одно значение NaN

nan_idx_q = np.where(df_new_data['query_flags'].isnull())[0]
nan_idx_c = np.where(df_new_data['caption'].isnull())[0]
nan_idx_n = np.where(df_new_data['image_name'].isnull())[0]

nan_idx_q = np.concatenate((nan_idx_q, nan_idx_c), axis=0)
for value in nan_idx_n:
    if value not in nan_idx_q:
        nan_idx_q = np.concatenate((nan_idx_q, nan_idx_n), axis=0)

In [278]:
nan_idx_q

array([], dtype=int64)

In [279]:
for idx in nan_idx_q:
    df_new_data.drop(idx,  axis=1,inplace=True)
    
for idx in nan_idx_q:
    df.drop(idx, inplace=True)

In [280]:
len(df)

1000

In [200]:
#датафрем с названием файла и его query_flags
dict_flags = dict(zip(list(df_new_data.image_name), list(df_new_data.query_flags)))

#список имен файлов для удаления
list_delete = []
for key,value in dict_flags.items():
    if value ==1:
        list_delete.append(key)

In [201]:
list_delete[:10]

['10006.jpg',
 '10015.jpg',
 '10016.jpg',
 '10019.jpg',
 '10024.jpg',
 '10025.jpg',
 '10031.jpg',
 '10035.jpg',
 '10036.jpg',
 '10045.jpg']

In [202]:
#удаление из главной таблицы строк, чьи изображения имеют query_flags==1
for idx in list_delete:
    index = df.index [df['image_name']== idx ].tolist()
    df.drop(index, inplace=True)

In [203]:
#удаление из новой таблицы строк, чьи изображения имеют query_flags==1
for idx in list_delete:
    index = df_new_data.index [df_new_data['image_name']== idx ].tolist()
    df_new_data.drop(index, inplace=True)

In [204]:
df.head()

,image_name,caption,width,height,url,LANGUAGE,similarity,punsafe,pwatermark,hash
0,10000.jpg,"Продажа дома п. Высокая Гора, ул Черноморская...",1024,768,http://intrum11-29.intrumnet.com/files/crm/pro...,ru,0.293233,0.000334,0.031824,8753873322058583918
1,10001.jpg,"Купить 10652_plus size, Сарафан для девочек СМ...",1200,1600,https://main-cdn.sbermegamarket.ru/hlr-system/...,ru,0.266255,0.079889,0.304476,476272511972184164
2,10002.jpg,В Астрахани состоится юбилейный гала-концерт ф...,1280,853,https://folc.ru/wp-content/uploads/2018/11/IMG...,ru,0.292562,0.815100,0.219494,6713797085172910306
3,10003.jpg,"Оформление аквариума, акваскейп",763,1024,https://i0.wp.com/praeclara.ru/wp-content/uplo...,ru,0.274981,0.000753,0.229119,-1069998755039171206
4,10004.jpg,"""""""Керамическая ваза для цветов ручной работы ...",720,720,https://images.satu.kz/80068858_keramicheskaya...,ru,0.278370,0.000036,0.167407,-1076175039013698791


In [205]:
len(df)

620

In [206]:
df_new_data.head()

,image_name,caption,query_flags
0,10000.jpg,Три кирпичных дома с забором и двумя соснами,0
1,10001.jpg,девочка в сарафане и белой блузке с черной баб...,0
2,10002.jpg,"танчующие девочки и мальчики в футболках ""я л...",0
3,10003.jpg,зеленое растение в прозрачном аквариуме,0
4,10004.jpg,черно- фиолетовая ваза с цветами,0


2. затем нужно заменить описания в основной цсвшке на описания из дополнительной для оставшихся фото. описания к фото записаны в колонке caption

In [ ]:
dict_caption = dict(zip(list(df_new_data.image_name), list(df_new_data.caption)))

In [ ]:
len(df)

In [ ]:
for i in list(df.index):
    line = df.loc[i]
    for name,new_caption in dict_caption.items():
        if line['image_name'] == name:
            df.at[i,'caption'] = dict_caption[name]

In [ ]:
df.head()

3. после этого на выходе вы получите несколько цсвшек, в которых исправленные описания и часть фото удалены. те фото, которых нет в цсвшках нужно удалить из самих архивов с фото

рекомендую к названию фото просто добавить номер папки, откуда это фото, в стиле 0.png станет 0_0.png, если это фото из нулевого тарника например. название самого фото меняем и название фото в цсвшке, соответственно

In [ ]:
def rename_jpg(x):
    x = x[:-4] + '_' + str(shard_idx) + '.jpg'
    return x

df['image_name'] = df['image_name'].apply(rename_jpg)

In [ ]:
df.head()

In [ ]:
new_csv_name = str(shard_idx) + '_new.csv'

path_save = os.path.join(dir_save,new_csv_name)
df.to_csv(path_save, index=False)

In [ ]:
tar_extr_path = 'C:/Users/annai/Practic/Filter_new_csv/tar_file'

In [ ]:
tar_name = str(shard_idx) + '.tar'
with tarfile.open(os.path.join(dir_name,tar_name)) as tar:
    tar.extractall(tar_extr_path)

In [ ]:
csv_names = df['image_name'].to_list()
for idx, name in enumerate(csv_names):
    csv_names[idx]=name[:-4] + '_' + str(shard_idx) + '.jpg'

In [ ]:
#удаляем те файлы, которых нет в нашей таблице, а те, что есть переименовываем с указанием шарда
count = 0
for filename in os.listdir(tar_extr_path):
    if '_' not in filename:
        new_name = filename[:-4] + '_' + str(shard_idx) + '.jpg'
        if new_name in csv_names:
            os.rename(os.path.join(tar_extr_path, filename), os.path.join(tar_extr_path,new_name))
        else:
            os.remove(os.path.join(tar_extr_path, filename))  
            
            count +=1

In [ ]:
print('delete {} images from {} shard'.format(count, shard_idx))

4. теперь нужно пересобрать цсвшки и тарники так, чтобы там было по 1000 фотографий. только у последней цсвшки в списке и у последнего соответствующего ему тарника может быть меньше 1000 фоток.


In [ ]:
csv_1 = 'C:/Users/annai/Filter_new_csv/rename/0_new.csv'
csv_2 = 'C:/Users/annai/Filter_new_csv/rename/3_new.csv'
csv_3 = 'C:/Users/annai/Filter_new_csv/rename/4_new.csv'

In [ ]:
csv_1 = pd.read_csv(csv_1)
csv_2 = pd.read_csv(csv_2)
csv_3 = pd.read_csv(csv_3)

In [ ]:
len(csv_1)

In [ ]:
len(csv_2)

In [ ]:
len(csv_3)

In [ ]:
itog_csv = {}
iter_sum = 0

for csv_name in filename:
    csv_df = pd.read_csv(os.path.join(dir_save,csv_name))
    
    if iter_sum + len(csv_df) < 1000:
        iter_sum = iter_sum + len(csv_df)
        itog_csv[csv_name] = len(csv_df)
    else:
        delta = 1000 - iter_sum
        itog_csv[csv_name] = delta

In [ ]:
itog_csv

In [ ]:
# create new final csv with merge other dataframe
i = 0
for k,v in itog_csv.items():
    i += 1
    data_i = pd.read_csv(os.path.join(dir_save,k))
    data_i = data_i.iloc[:v]
    if i==1:
        concate_data = data_i
    else:
        concate_data = pd.concat([concate_data,data_i])


In [ ]:
final_shard = 0
final_csv_name = str(final_shard) + '_final.csv'
final_path_save = os.path.join(dir_save,final_csv_name)
concate_data.to_csv(final_path_save, index=False)

In [ ]:
concate_data.head()

In [ ]:
def create_tar(img_path, tar_path,concate_data)
    csv_names = concate_data['image_name'].to_list()
    os.chdir('C:/Users/annai/Filter_new_csv/tar_file')

    arc_name = str(0) + '.tar'
    tar_export = 'C:/Users/annai/Filter_new_csv'
    fp=tarfile.open(os.path.join(tar_export, arc_name), mode='w')
    try: 
        for file in glob.glob('*.*'):
            if file in csv_names:
                fp.add(file)
    finally:
        fp.close()
    

# Main loop

In [47]:
def get_df(idx, dir_name): #по индексу получить датафремы для главной таблицы и новой
    # print(idx)
    file_data = str(idx) + '.csv'
    new_data = str(idx) + '_new.csv'
    csv_path = os.path.join(dir_name,file_data)
    new_data_path = os.path.join(dir_name,new_data)
    df = pd.read_csv(csv_path)
    print(new_data_path)
    if idx == str(21):
        # print('cheak 21')
        # df_new_data = pd.read_csv(new_data_path, sep=';',encoding='cp1251', on_bad_lines='skip') #30-59 shard for 0
        df_new_data = pd.read_csv(new_data_path, sep = ';', on_bad_lines='skip')  
    else:
        df_new_data = pd.read_csv(new_data_path, sep = ',', on_bad_lines='skip')
    return df, df_new_data

In [30]:
def idx_nan(df_new_data): #получить индекс строки, где хотя бы одно значение NaN
    
    nan_idx_q = np.where(df_new_data['query_flags'].isnull())[0]
    nan_idx_c = np.where(df_new_data['caption'].isnull())[0]
    nan_idx_n = np.where(df_new_data['image_name'].isnull())[0]

    nan_idx_q = np.concatenate((nan_idx_q, nan_idx_c), axis=0)
    for value in nan_idx_n:
        if value not in nan_idx_q:
            nan_idx_q = np.concatenate((nan_idx_q, nan_idx_n), axis=0)
    return nan_idx_q

In [31]:
def del_from_df(list_delete, df):#удаление из главной таблицы строк, чьи изображения имеют query_flags==1
    for idx in list_delete:
        index = df.index [df['image_name']== idx ].tolist()
        df.drop(index, inplace=True)
    return(df)

In [32]:
def rename_jpg(x):
    x = x[:-4] + '_' + str(shard_idx) + '.jpg'
    return x

In [33]:
# def update_caption(x):
#     x = x[:-4] + '_' + str(shard_idx) + '.jpg'
#     return x

In [34]:
def df_preproces(index,dir_name):
    
    #по индексу получить датафремы для главной таблицы и новой
    df, df_new_data = get_df(index, dir_name)
    #получить индекс строки, где хотя бы одно значение NaN
    nan_idx= idx_nan(df_new_data)

    #выбросить эти строки из основной и новой таблицы,чтобы не мешались
    for idx in nan_idx:
        df_new_data.drop(idx, inplace=True)
    for idx in nan_idx:
        df.drop(idx, inplace=True)

    #датафрем с названием файла и его query_flags
    dict_flags = dict(zip(list(df_new_data.image_name), list(df_new_data.query_flags)))
    #список имен файлов для удаления
    list_delete = []
    for key,value in dict_flags.items():
        if value ==1:
            list_delete.append(key)

    #удаление из  таблицы строк, чьи изображения имеют query_flags==1
    df = del_from_df(list_delete, df)
    df_new_data = del_from_df(list_delete, df_new_data)

    #датафрем с названием файла и его caption
    dict_caption = dict(zip(list(df_new_data.image_name), list(df_new_data.caption)))
    # замена caption в главной таблице
    for i in list(df.index):
        line = df.loc[i]
        for name,new_caption in dict_caption.items():
            if line['image_name'] == name:
                df.at[i,'caption'] = dict_caption[name]
    #замена названия файла с учетом номера оригинального шарда
    df['image_name'] = df['image_name'].apply(rename_jpg)
    
    return df

In [35]:
def delete_jpg(tar_extr_path, df, shard_idx): #удаляем те файлы, которых нет в нашей таблице, а те, что есть переименовываем с указанием шарда
    
    #получаем список имен из главной таблицы
    csv_names = df['image_name'].to_list()
    count = 0
    #сравниваем каждый файл в папке со списком нужных имен из csv
    for filename in os.listdir(tar_extr_path):
        if '_' not in filename: #если нет '_' значит, это только что разархивированные фото
            new_name = filename[:-4] + '_' + str(shard_idx) + '.jpg'
            if new_name in csv_names:
                os.rename(os.path.join(tar_extr_path, filename), os.path.join(tar_extr_path,new_name))
            else:
                os.remove(os.path.join(tar_extr_path, filename))  
                count +=1
    return count

In [36]:
def merge_csv(index_new_df, dir_save, delta, past_delta): #объединить уже существующие таблицы
    
    #delta = filies_count - 1000 
    itog_csv = {}
    iter_sum = 0
    last_csv = index_new_df[-1]
    first_csv = index_new_df[0]
    
    if past_delta ==0:
        for i in index_new_df:
            csv_name = str(i) + '_update.csv'
            path_save = os.path.join(dir_save,csv_name)
            csv_df = pd.read_csv(path_save)
            if i != last_csv:
                itog_csv[csv_name] = len(csv_df)
            else:
                itog_csv[csv_name] = len(csv_df) - delta
                
    else:
          for i in index_new_df:
            csv_name = str(i) + '_update.csv'
            path_save = os.path.join(dir_save,csv_name)
            csv_df = pd.read_csv(path_save)
            
            if i == first_csv:
                itog_csv[csv_name] = past_delta
            elif i != last_csv:
                itog_csv[csv_name] = len(csv_df)
            else:
                itog_csv[csv_name] = len(csv_df) - delta       
    print(itog_csv)

    first_folder_csv = '0_update.csv'
    # create new final csv with merge other dataframe
    i = 0
    dict_len = len(itog_csv)
    for k,v in itog_csv.items():
        # print(i)
        # print(k)

        if delta != 0:
            if k == first_folder_csv: #когда первая таблица из архива
                # print('первая таблица из архива. start_value = 0')
                data_i = pd.read_csv(os.path.join(dir_save,k))
                data_split = data_i.iloc[0:v]
                # print(len(data_split))
                # data_split.to_csv('C:/Users/annai/Practic/Filter_new_csv/test0.csv', index=False)
                concate_data = data_split
                i += 1
                # print('------------------')
            
            elif i==0: #когда начинается с остатка
                
                # print('start_value = delta')
                data_i = pd.read_csv(os.path.join(dir_save,k))
                data_split = data_i.iloc[len(data_i) - past_delta:len(data_i) - past_delta+v]
                # print(len(data_split))
                # data_split.to_csv('C:/Users/annai/Practic/Filter_new_csv/test10_2.csv', index=False)
                concate_data = data_split
                i += 1
                # print('------------------')
                      
            else:
                # print('start_value = 0')
                data_i = pd.read_csv(os.path.join(dir_save,k))
                data_split = data_i.iloc[0:v]
                # print(len(data_split))
                # data_split.to_csv('C:/Users/annai/Practic/Filter_new_csv/test10_1.csv', index=False)
                i += 1
                concate_data = pd.concat([concate_data,data_split])
                # print('------------------')
        else:
            # print('нет остатка. Таблицы полностью. start_value = 0')
            data_i = pd.read_csv(os.path.join(dir_save,k))
            data_0 = data_i.iloc[0:v]
            # data_0.to_csv('C:/Users/annai/Practic/Filter_new_csv/test10_1.csv', index=False)
            i += 1
            # print('------------------')
    
    return concate_data

In [37]:
def create_tar(img_path, tar_path,concate_data, final_idx):
    csv_names = concate_data['image_name'].to_list()
    os.chdir(img_path)

    tar_name = str(final_idx) + '.tar'
    tar_path = os.path.join(tar_path, tar_name)
    fp=tarfile.open(tar_path, mode='w')
    try: 
        for file in glob.glob('*.*'):
            if file in csv_names:
                fp.add(file)
    finally:
        fp.close()
        
    print('export new tar', tar_path) 
    shutil.move(tar_path, 'C:/Users/annai/Practic/Filter_new_csv')
    
    #удалить эти файлы, чтобы не мешались
    for file in (os.listdir(img_path)):
        if file in csv_names:
            os.remove(file)

In [38]:
filename = os.listdir(dir_name)
print(filename)

['0.csv', '0.tar', '0_new.csv', '1.csv', '1.tar', '10.csv', '10.tar', '10_new.csv', '11.csv', '11.tar', '11_new.csv', '12.csv', '12.tar', '12_new.csv', '13.csv', '13.tar', '13_new.csv', '14.csv', '14.tar', '14_new.csv', '15.csv', '15.tar', '16.csv', '16.tar', '16_new.csv', '17.csv', '17.tar', '17_new.csv', '18.csv', '18.tar', '18_new.csv', '19.csv', '19.tar', '2.csv', '2.tar', '20.csv', '20.tar', '21.csv', '21.tar', '21_new.csv', '22.csv', '22.tar', '22_new.csv', '23.csv', '23.tar', '23_new.csv', '24.csv', '24.tar', '24_new.csv', '25.csv', '25.tar', '26.csv', '26.tar', '27.csv', '27.tar', '27_new.csv', '28.csv', '28.tar', '28_new.csv', '29.csv', '29.tar', '29_new.csv', '3.csv', '3.tar', '30.csv', '30.tar', '30_new.csv', '4.csv', '4.tar', '4_new.csv', '5.csv', '5.tar', '5_new.csv', '6.csv', '6.tar', '6_new.csv', '7.csv', '7.tar', '7_new.csv', '8.csv', '8.tar', '8_new.csv', '9.csv', '9.tar', '9_new.csv']


In [ ]:
final_idx = 0
dir_name = 'C:/Users/annai/Filter_new_csv/laion_data_0_29'
dir_save = 'C:/Users/annai/Filter_new_csv/rename'
tar_extr_path = 'C:/Users/annai/Filter_new_csv/tar_file'

filename = os.listdir(dir_name)
#iter_sum = 0 #для проверки кол-ва файлов для отправки в s3
index_new_df = []
past_delta = 0

for i, name in enumerate(filename):
    if '.csv' in name:
        if 'new' not in name:
            print('choose new csv:', name)
            shard_idx = name.split('.')[0]
            csv_new_name = shard_idx + '_new.csv'
            if csv_new_name in filename:
                print('у этого шарда есть new_csv')
                #получить обновленную таблицу с учетом новых данных
                df_i = df_preproces(shard_idx,dir_name) 
                index_new_df.append(shard_idx)
                
                #и сохраняем
                new_csv_name = str(shard_idx) + '_update.csv'
                path_csv_save = os.path.join(dir_save,new_csv_name)
                df_i.to_csv(path_csv_save, index=False)
                print('shard_idx', shard_idx)
                #разархивировать
                tar_name = str(shard_idx) + '.tar'
                print('tar_name', tar_name)
                with tarfile.open(os.path.join(dir_name,tar_name)) as tar:
                    tar.extractall(tar_extr_path)
                #удаляем те файлы, которых нет в нашей таблице, а те, что есть переименовываем с указанием шарда    
                count = delete_jpg(tar_extr_path,df_i,shard_idx)
                
            else:
                print('у этого шарда нет new_csv')
                if shard_idx not in index_new_df:
                    index_new_df.append(shard_idx)
                    csv_path = os.path.join(dir_name,name)
                    df_i = pd.read_csv(csv_path, sep = ',', on_bad_lines='skip')
                    df_i= df_i.dropna()
                    #замена названия файла с учетом номера оригинального шарда
                    df_i['image_name'] = df_i['image_name'].apply(rename_jpg)
                    
                    #и сохраняем
                    new_csv_name = str(shard_idx) + '_update.csv'
                    path_csv_save = os.path.join(dir_save,new_csv_name)
                    df_i.to_csv(path_csv_save, index=False)
                    print('shard_idx', shard_idx)
                    #разархивировать
                    tar_name = str(shard_idx) + '.tar'
                    print('tar_name', tar_name)
                    with tarfile.open(os.path.join(dir_name,tar_name)) as tar:
                        tar.extractall(tar_extr_path)
                        
                    #сравниваем каждый файл в папке со списком нужных имен из csv
                    for filename in os.listdir(tar_extr_path):
                        if '_' not in filename: #если нет '_' значит, это только что разархивированные фото
                            new_name = filename[:-4] + '_' + str(shard_idx) + '.jpg'
                            os.rename(os.path.join(tar_extr_path, filename), os.path.join(tar_extr_path,new_name))

            #получаем кол-во результирующих файлов для отправки в s3
            shard_idx = len(os.listdir(tar_extr_path))
            print('проверяем кол-во вайлов в папке',shard_idx )
            if shard_idx > 1000:
                print('стало > 1000')
                print(index_new_df)
                #объединить уже существующие таблицы
                delta = shard_idx - 1000
                concate_data = merge_csv(index_new_df, dir_save, delta, past_delta)
                past_delta = delta
                last = index_new_df[-1] #сохраняем только последнюю таблицу, тк из нее не все файлы были взяты
                index_new_df = []
                index_new_df.append(last)

                #сохранить эту таблицу
                final_csv_name = str(final_idx) + '_final.csv'
                final_path_save = os.path.join(dir_save,final_csv_name)
                concate_data.to_csv(final_path_save, index=False)
                print('export final csv', final_path_save)
                #получаем список имен из новой таблицы
                csv_names = concate_data['image_name'].to_list()
                #используя его, архивируем 1000 изображений
                tar_path = 'C:/Users/annai/Practic/Filter_new_csv/tar_file'
                create_tar(tar_extr_path, tar_path,concate_data, final_idx )
                final_idx += 1
                print('---------------------------')     
            else:
                print('все еще < 1000')
                print('---------------------------')
                continue
        
        
            

In [49]:
# работа с остатком фото
idx = 9
file_data = str(idx) + '_update.csv'
csv_path = os.path.join(dir_save,file_data)
df = pd.read_csv(csv_path)
 #получаем список имен из главной таблицы
csv_names = df['image_name'].to_list()
count = 0
#сравниваем каждый файл в папке со списком нужных имен из csv
for filename in os.listdir(tar_extr_path):
        if new_name in csv_names:
            continue
        else:
            os.remove(os.path.join(tar_extr_path, filename))  

In [ ]:
final_idx = 29
tar_name = str(final_idx) + '.tar'
tar_path = os.path.join(tar_path, tar_name)
fp=tarfile.open(tar_path, mode='w')
try: 
    for file in glob.glob('*.*'):
        if file in csv_names:
            fp.add(file)
finally:
    fp.close()

print('export new tar', tar_path) 
shutil.move(tar_path, 'C:/Users/annai/_Practic/Filter_new_csv')

#удалить эти файлы, чтобы не мешались
for file in (os.listdir(tar_extr_path)):
    if file in csv_names:
        os.remove(file)

In [51]:
data_i = pd.read_csv(os.path.join(dir_save,file_data))
v = 158
data_split = data_i.iloc[v:len(data_i)]
#сохранить эту таблицу
final_csv_name = str(final_idx) + '_final.csv'
final_path_save = os.path.join(dir_save,final_csv_name)
data_split.to_csv(final_path_save, index=False)

## Upload to S3import io
import re
import tarfile
import boto3

In [302]:
import io
import re
import tarfile
import boto3

In [303]:
BUCKET_NAME = 'officecds-bucket01'
s3 = boto3.client(
    "s3",
    aws_access_key_id='',
    aws_secret_access_key='',
    endpoint_url=''
)

In [ ]:
#upload csv
csv_path = 'C:/Users/annai/Filter_new_csv/rename'
for name in (os.listdir(csv_path)):
    if 'final.csv' in name:
        path = os.path.join(csv_path, name)
        print(path)
        path_upload = '/datasets_v3/text_image/rus_tematic/laion_data_0_59/shard_filter/' + name
        print(path_upload)
        with open(path,'rb') as f:
            s3.upload_fileobj(f, BUCKET_NAME, path_upload)

In [ ]:
#upload tar
path = 'C:/Users/annai/Filter_new_csv/'
tar_list = []
for name in (os.listdir(path)):
    if '.tar' in name:
        tar_list.append(name)
        
for name in tar_list:
    path2 = path + name
    path_upload = '/datasets_v3/text_image/rus_tematic/data_0_59/shard_filter/' + name
    print(path_upload)
    with open(path,'rb') as tar:
        s3.upload_fileobj(tar, BUCKET_NAME, path_upload)

In [327]:
for i in range(3,29):
    name = str(i) +'.tar'
    path = 'C:/Users/annai/Practic/Filter_new_csv/' + name
    path_upload = '/datasets_v3/text_image/rus_tematic/laion_data_0_59/shard_filter/' + name
    with open(path,'rb') as tar:
        s3.upload_fileobj(tar, BUCKET_NAME, path_upload)